## Amazon Sales 
### File preparation


In [2]:
import pyodbc
import pandas as pd

In [3]:
# Set up your connection string

conn_str = (
    'DRIVER={SQL Server};'
    'SERVER=DESKTOP-0MPFAM1;'
    'DATABASE=Amazon Sales Report;'
    'Trusted_Connection=True'
)

In [4]:
# Establish connection
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

#### StatusDIM Table

In [5]:
query = "select status, ROW_NUMBER() OVER (ORDER BY [Status]) as statusID from SalesOrderDetails group by status"

# Use pandas to execute the query and store the result in a DataFrame
df_status = pd.read_sql(query, conn)

df_status


C:\Users\Admin\AppData\Local\Temp\ipykernel_3092\2793704294.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_status = pd.read_sql(query, conn)


,status,statusID
0,Cancelled,1
1,Pending,2
2,Pending - Waiting for Pick Up,3
3,Shipped,4
4,Shipped - Damaged,5
5,Shipped - Delivered to Buyer,6
6,Shipped - Lost in Transit,7
7,Shipped - Out for Delivery,8
8,Shipped - Picked Up,9
9,Shipped - Rejected by Buyer,10


In [6]:
# Save the result to a CSV file
df_status.to_csv("files\status_output.csv", index=False)

#### ProductsDIM Table

In [7]:
# rename ProductID to ProductAltID act as business key and set ProductID using row_number() to be a surgute key
query2 = """               
    SELECT DISTINCT ProductID AS ProductAltID,
    [ProductName],
    [DesignNo ],
    [CategoryID],
    [Size],[Color],
    [Price], 
    ROW_NUMBER() OVER (ORDER BY ProductID) as ProductID 
    FROM [Amazon Sales Report].[dbo].[Products]
"""
# Use pandas to execute the query and store the result in a DataFrame
df_product = pd.read_sql(query2, conn)

C:\Users\Admin\AppData\Local\Temp\ipykernel_3092\2664374736.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_product = pd.read_sql(query2, conn)


In [8]:
# Save the result to a CSV file
df_product.to_csv("files\product_output.csv", index=False)

#### customerDIM table

In [9]:
query3 = """
 SELECT DISTINCT [CustomerID] AS CustomerAltID
    ,[FirstName]
    ,[LastName]
    ,[Gender]
    ,[Email]
    ,[PhoneNumber]
    ,[FirstLine]
    ,[SecondLine]
    ,[District]
    ,[City]
    ,[Country]
    ,[SignupDate]
    ,ROW_NUMBER() OVER (ORDER BY [CustomerID]) as CustomerID
    FROM [Amazon Sales Report].[dbo].[Customers]
    """

# Use pandas to execute the query and store the result in a DataFrame
df_customer = pd.read_sql(query3, conn)

C:\Users\Admin\AppData\Local\Temp\ipykernel_3092\1170833425.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_customer = pd.read_sql(query3, conn)


In [10]:
# Save the result to a CSV file
df_customer.to_csv("files\customer_output.csv", index=False)

#### InventoryDIM Table

In [11]:
query4 = """
    SELECT distinct [InventoryID ] AS InventoryAltID
    ,[ProductID]
    ,[OnShelfQuantity]
    ,[WarehouseID] 
    ,ROW_NUMBER() OVER (ORDER BY [InventoryID]) as InventoryID
    FROM [Amazon Sales Report].[dbo].[Inventories]
    """

# Use pandas to execute the query and store the result in a DataFrame
df_Inventory = pd.read_sql(query4, conn)

C:\Users\Admin\AppData\Local\Temp\ipykernel_3092\3422400751.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_Inventory = pd.read_sql(query4, conn)


In [12]:
# Save the result to a CSV file
df_Inventory.to_csv("files\Inventory_output.csv", index=False)

#### warehouseDIM Table

In [13]:
query5 = """
    SELECT [warehouseID] as WarehouseAltID
    ,[warehouseName]
    ,[locationCity]
    ,[locationRegion]
    ,[address]
    ,[capacity]
    ,[Phone]
    ,[managerName]
    ,[operationalStatus]
    ,ROW_NUMBER() OVER (ORDER BY [warehouseID]) as warehouseID
    FROM [Amazon Sales Report].[dbo].[Warehouses]
    """

# Use pandas to execute the query and store the result in a DataFrame
df_Warehouse = pd.read_sql(query5, conn)

C:\Users\Admin\AppData\Local\Temp\ipykernel_3092\3567969099.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_Warehouse = pd.read_sql(query5, conn)


In [14]:
# Save the result to a CSV file
df_Warehouse.to_csv("files\Warehouse_output.csv", index=False)

#### FulfilmentDIM Table

In [15]:
query6 = """
    SELECT [Fulfilment],
    CAST(ROW_NUMBER() OVER (ORDER BY [Fulfilment]) AS INT) as FulfilmentID 
    from SalesOrderDetails 
    group by [Fulfilment]
    """

# Use pandas to execute the query and store the result in a DataFrame
df_Fulfilment = pd.read_sql(query6, conn)

C:\Users\Admin\AppData\Local\Temp\ipykernel_3092\1653474787.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_Fulfilment = pd.read_sql(query6, conn)


In [16]:
# Save the result to a CSV file
df_Fulfilment.to_csv("files\Fulfilment_output.csv", index=False)

#### Channel_ShipServiceDIM Table

In [17]:
query7 = """
    SELECT DISTINCT [Sales_Channel ], [ship_service_level]
    FROM [Amazon Sales Report].[dbo].[SalesOrderDetails]
    """

# Use pandas to execute the query and store the result in a DataFrame
df_Channel_ShipService = pd.read_sql(query7, conn)

df_Channel_ShipService['Channel_ShipServiceID'] = range(1, len(df_Channel_ShipService) + 1)

C:\Users\Admin\AppData\Local\Temp\ipykernel_3092\2796003986.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_Channel_ShipService = pd.read_sql(query7, conn)


In [18]:
# Save the result to a CSV file
df_Channel_ShipService.to_csv("files\Channel_ShipService_output.csv", index=False)

#### FactSales Table


In [19]:
query8 = """
    SELECT *
    FROM [Amazon Sales Report].[dbo].[SalesOrderDetails]
    """

# Use pandas to execute the query and store the result in a DataFrame
df_fact = pd.read_sql(query8, conn)

df_fact.info()


C:\Users\Admin\AppData\Local\Temp\ipykernel_3092\424619199.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_fact = pd.read_sql(query8, conn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128703 entries, 0 to 128702
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   OrderLineID         128703 non-null  int64  
 1   OrderID             128703 non-null  int64  
 2   OrderAltKey         128703 non-null  object 
 3   Date                128703 non-null  object 
 4   Status              128703 non-null  object 
 5   Fulfilment          128703 non-null  object 
 6   Sales_Channel       128703 non-null  object 
 7   ship_service_level  128703 non-null  object 
 8   ProductID           128703 non-null  int64  
 9   Category            128703 non-null  object 
 10  Size                128703 non-null  object 
 11  ASIN                128703 non-null  object 
 12  Courier_Status      128703 non-null  object 
 13  Qty                 128703 non-null  int64  
 14  currency            128703 non-null  object 
 15  UnitPrice           128703 non-nul

In [20]:
df_fact.head(5)

,OrderLineID,OrderID,OrderAltKey,Date,Status,Fulfilment,Sales_Channel,ship_service_level,ProductID,Category,...,ASIN,Courier_Status,Qty,currency,UnitPrice,SubTotal,ship_country,fulfilledBy,B2B,promotions_used
0,1,1,408-9483921-6621140,2024-07-02,Shipped,Amazon,Amazon.in,Expedited,1912,SET,...,B09M6T8JXP,Shipped,1,EGP,4599,4599.0,EG,Easy Ship,FALSE,
1,2,2,408-9377454-4728330,2024-07-02,Shipped,Amazon,Amazon.in,Expedited,1541,KURTA,...,B08QGLBQGM,Shipped,1,EGP,1214,1214.0,EG,Easy Ship,FALSE,"""Amazon PLCC Free-Financing Universal Merchant..."
2,3,3,408-9103661-4310737,2024-07-02,Shipped,Amazon,Amazon.in,Expedited,1230,TOP,...,B08RYPWC3J,Shipped,1,EGP,804,804.0,EG,,TRUE,IN Core Free Shipping 2015/04/08 23-48-5-108
3,4,4,408-8949109-9614720,2024-07-02,Shipped,Amazon,Amazon.in,Expedited,7143,KURTA,...,B09LD3TCYP,Shipped,1,EGP,1399,1399.0,EG,Easy Ship,FALSE,
4,5,5,408-8834155-2110755,2024-07-02,Shipped,Amazon,Amazon.in,Expedited,1270,KURTA SET,...,B08V1R5MMB,Shipped,1,EGP,3499,3499.0,EG,,FALSE,


In [21]:
# Merge df_fact with df_status based on the 'Status' field from df_fact and 'status' from df_status
df_merged_status = pd.merge(df_fact, df_status, how='left', left_on='Status', right_on='status')

# Drop the original 'Status' and 'status' columns (keeping the 'statusID' column)
df_merged_status.drop(columns=['Status', 'status'], inplace=True)

# Rename 'statusID' to 'StatusID' for clarity
df_merged_status.rename(columns={'statusID': 'StatusID'}, inplace=True)

# Display the final result
df_merged_status.head(5)

,OrderLineID,OrderID,OrderAltKey,Date,Fulfilment,Sales_Channel,ship_service_level,ProductID,Category,Size,...,Courier_Status,Qty,currency,UnitPrice,SubTotal,ship_country,fulfilledBy,B2B,promotions_used,StatusID
0,1,1,408-9483921-6621140,2024-07-02,Amazon,Amazon.in,Expedited,1912,SET,XL,...,Shipped,1,EGP,4599,4599.0,EG,Easy Ship,FALSE,,4
1,2,2,408-9377454-4728330,2024-07-02,Amazon,Amazon.in,Expedited,1541,KURTA,L,...,Shipped,1,EGP,1214,1214.0,EG,Easy Ship,FALSE,"""Amazon PLCC Free-Financing Universal Merchant...",4
2,3,3,408-9103661-4310737,2024-07-02,Amazon,Amazon.in,Expedited,1230,TOP,XL,...,Shipped,1,EGP,804,804.0,EG,,TRUE,IN Core Free Shipping 2015/04/08 23-48-5-108,4
3,4,4,408-8949109-9614720,2024-07-02,Amazon,Amazon.in,Expedited,7143,KURTA,6XL,...,Shipped,1,EGP,1399,1399.0,EG,Easy Ship,FALSE,,4
4,5,5,408-8834155-2110755,2024-07-02,Amazon,Amazon.in,Expedited,1270,KURTA SET,M,...,Shipped,1,EGP,3499,3499.0,EG,,FALSE,,4


In [22]:
# Merge df_merged_status with df_Fulfilment based on the 'Fulfilment' field from df_merged_status and 'Fulfilment' from df_Fulfilment
df_merged_Fulfilment = pd.merge(df_merged_status, df_Fulfilment, how='left', left_on='Fulfilment', right_on='Fulfilment')

# Drop the original 'Fulfilment' and 'Fulfilment' columns (keeping the 'FulfilmentID' column)
df_merged_Fulfilment.drop(columns=['Fulfilment', 'Fulfilment'], inplace=True)

df_merged_Fulfilment.head()

,OrderLineID,OrderID,OrderAltKey,Date,Sales_Channel,ship_service_level,ProductID,Category,Size,ASIN,...,Qty,currency,UnitPrice,SubTotal,ship_country,fulfilledBy,B2B,promotions_used,StatusID,FulfilmentID
0,1,1,408-9483921-6621140,2024-07-02,Amazon.in,Expedited,1912,SET,XL,B09M6T8JXP,...,1,EGP,4599,4599.0,EG,Easy Ship,FALSE,,4,1
1,2,2,408-9377454-4728330,2024-07-02,Amazon.in,Expedited,1541,KURTA,L,B08QGLBQGM,...,1,EGP,1214,1214.0,EG,Easy Ship,FALSE,"""Amazon PLCC Free-Financing Universal Merchant...",4,1
2,3,3,408-9103661-4310737,2024-07-02,Amazon.in,Expedited,1230,TOP,XL,B08RYPWC3J,...,1,EGP,804,804.0,EG,,TRUE,IN Core Free Shipping 2015/04/08 23-48-5-108,4,1
3,4,4,408-8949109-9614720,2024-07-02,Amazon.in,Expedited,7143,KURTA,6XL,B09LD3TCYP,...,1,EGP,1399,1399.0,EG,Easy Ship,FALSE,,4,1
4,5,5,408-8834155-2110755,2024-07-02,Amazon.in,Expedited,1270,KURTA SET,M,B08V1R5MMB,...,1,EGP,3499,3499.0,EG,,FALSE,,4,1


In [23]:
# Merge df_merged_Fulfilment with df_order  to set customerID and order total price
query9 = """
    SELECT *
    FROM [Amazon Sales Report].[dbo].[Orders]
    """
df_order = pd.read_sql(query9, conn)
df_order

df_customer = pd.merge(df_merged_Fulfilment, df_order, how='left', left_on='OrderID', right_on='OrderID')

# Drop the 'OrderID', 'OrderAltKey',and 'OrderDate' columns (keeping the customerID and totalPrice column)
df_customer.drop(columns=['OrderAltKey_x', 'OrderAltKey_y', 'OrderDate'], inplace=True)

df_customer.head()


C:\Users\Admin\AppData\Local\Temp\ipykernel_3092\3226040750.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_order = pd.read_sql(query9, conn)


,OrderLineID,OrderID,Date,Sales_Channel,ship_service_level,ProductID,Category,Size,ASIN,Courier_Status,...,UnitPrice,SubTotal,ship_country,fulfilledBy,B2B,promotions_used,StatusID,FulfilmentID,CustomerID,TotalPrice
0,1,1,2024-07-02,Amazon.in,Expedited,1912,SET,XL,B09M6T8JXP,Shipped,...,4599,4599.0,EG,Easy Ship,FALSE,,4,1,100939,4599.0
1,2,2,2024-07-02,Amazon.in,Expedited,1541,KURTA,L,B08QGLBQGM,Shipped,...,1214,1214.0,EG,Easy Ship,FALSE,"""Amazon PLCC Free-Financing Universal Merchant...",4,1,100983,1214.0
2,3,3,2024-07-02,Amazon.in,Expedited,1230,TOP,XL,B08RYPWC3J,Shipped,...,804,804.0,EG,,TRUE,IN Core Free Shipping 2015/04/08 23-48-5-108,4,1,100389,804.0
3,4,4,2024-07-02,Amazon.in,Expedited,7143,KURTA,6XL,B09LD3TCYP,Shipped,...,1399,1399.0,EG,Easy Ship,FALSE,,4,1,100313,1399.0
4,5,5,2024-07-02,Amazon.in,Expedited,1270,KURTA SET,M,B08V1R5MMB,Shipped,...,3499,3499.0,EG,,FALSE,,4,1,100926,5994.0


In [24]:
# Merge df_customer with inventory based on the 'productID' field from df_customer and 'productID' from df_inventory
df_merged_inventory = pd.merge(df_customer, df_Inventory, how='left', left_on='ProductID', right_on='ProductID')

# Drop the original 'Fulfilment' and 'Fulfilment' columns (keeping the 'FulfilmentID' column)
df_merged_inventory.drop(columns=['InventoryAltID', 'OnShelfQuantity', 'WarehouseID'], inplace=True)
			

df_merged_inventory.head()

,OrderLineID,OrderID,Date,Sales_Channel,ship_service_level,ProductID,Category,Size,ASIN,Courier_Status,...,SubTotal,ship_country,fulfilledBy,B2B,promotions_used,StatusID,FulfilmentID,CustomerID,TotalPrice,InventoryID
0,1,1,2024-07-02,Amazon.in,Expedited,1912,SET,XL,B09M6T8JXP,Shipped,...,4599.0,EG,Easy Ship,FALSE,,4,1,100939,4599.0,1912
1,2,2,2024-07-02,Amazon.in,Expedited,1541,KURTA,L,B08QGLBQGM,Shipped,...,1214.0,EG,Easy Ship,FALSE,"""Amazon PLCC Free-Financing Universal Merchant...",4,1,100983,1214.0,1541
2,3,3,2024-07-02,Amazon.in,Expedited,1230,TOP,XL,B08RYPWC3J,Shipped,...,804.0,EG,,TRUE,IN Core Free Shipping 2015/04/08 23-48-5-108,4,1,100389,804.0,1230
3,4,4,2024-07-02,Amazon.in,Expedited,7143,KURTA,6XL,B09LD3TCYP,Shipped,...,1399.0,EG,Easy Ship,FALSE,,4,1,100313,1399.0,7143
4,5,5,2024-07-02,Amazon.in,Expedited,1270,KURTA SET,M,B08V1R5MMB,Shipped,...,3499.0,EG,,FALSE,,4,1,100926,5994.0,1270


In [25]:
# Merge df_fact with df_Channel_ShipService on 'Sales_Channel' and 'ship_service_level'
df_merged_Channel_ShipService = pd.merge(df_merged_inventory, df_Channel_ShipService, how='left', on=['Sales_Channel ', 'ship_service_level'])

# Drop the original 'Sales_Channel' and 'ship_service_level' columns
df_merged_Channel_ShipService.drop(columns=['Sales_Channel ', 'ship_service_level'], inplace=True)

df_merged_Channel_ShipService.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128703 entries, 0 to 128702
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   OrderLineID            128703 non-null  int64  
 1   OrderID                128703 non-null  int64  
 2   Date                   128703 non-null  object 
 3   ProductID              128703 non-null  int64  
 4   Category               128703 non-null  object 
 5   Size                   128703 non-null  object 
 6   ASIN                   128703 non-null  object 
 7   Courier_Status         128703 non-null  object 
 8   Qty                    128703 non-null  int64  
 9   currency               128703 non-null  object 
 10  UnitPrice              128703 non-null  object 
 11  SubTotal               128703 non-null  float64
 12  ship_country           128703 non-null  object 
 13  fulfilledBy            128703 non-null  object 
 14  B2B                    128703 non-nu

In [27]:
df_factSales = df_merged_Channel_ShipService[['OrderLineID', 'OrderID', 'Date', 'StatusID', 'FulfilmentID', 'Channel_ShipServiceID', 'ProductID','InventoryID', 'CustomerID', 'ASIN',
                      'Qty', 'SubTotal','TotalPrice', 'currency', 'ship_country', 'B2B', 'promotions_used']]

# Convert the Date column to the desired int format
df_factSales['Date'] = df_factSales['Date'].str.replace('-', '').astype(int)

C:\Users\Admin\AppData\Local\Temp\ipykernel_3092\1558965643.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_factSales['Date'] = df_factSales['Date'].str.replace('-', '').astype(int)


In [29]:
# Save the result to a CSV file
df_factSales.to_csv('factSales.csv', index=False)

#### CalenderDIM


In [31]:
query10 = """
SELECT 
    MIN([Date]) AS start_date, 
    MAX([Date]) AS end_date
FROM [Amazon Sales Report].[dbo].[SalesOrderDetails];
"""
date_range = pd.read_sql(query10, conn)

# Extract the start and end dates
start_date = date_range['start_date'][0]
end_date = date_range['end_date'][0]

# Function to create a date dimension
def create_date_dim(start_date, end_date):
    date_range = pd.date_range(start=start_date, end=end_date)
    df_date_dim = pd.DataFrame({'FullDate': date_range})
    
    # Add date-related columns
    df_date_dim['DateKey'] = df_date_dim['FullDate'].dt.strftime('%Y%m%d').astype(int)
    df_date_dim['Year'] = df_date_dim['FullDate'].dt.year
    df_date_dim['Month'] = df_date_dim['FullDate'].dt.month
    df_date_dim['Day'] = df_date_dim['FullDate'].dt.day
    df_date_dim['Quarter'] = df_date_dim['FullDate'].dt.quarter
    df_date_dim['WeekOfYear'] = df_date_dim['FullDate'].dt.isocalendar().week
    df_date_dim['DayOfYear'] = df_date_dim['FullDate'].dt.dayofyear
    df_date_dim['DayOfWeek'] = df_date_dim['FullDate'].dt.dayofweek + 1
    
    
    return df_date_dim

# Create the Date Dimension using the date range
df_date_dim = create_date_dim(start_date, end_date)


C:\Users\Admin\AppData\Local\Temp\ipykernel_3092\3610502049.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  date_range = pd.read_sql(query10, conn)


In [32]:
df_date_dim.to_csv('DateDim.csv', index=False)


In [33]:
# Close the connection
conn.close()